# **1. Librerías**

In [ ]:
# Para poder acceder a un archivo en Google Drive
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive

!pip install SimpleITK
!pip install timm
!pip install torch-geometric

# Obtención del dataset
%cd /content/drive/My Drive/Bivlab/Datasets/PICAI_Consolidado/Copia_de_Consolidado_/

Mounted at /content/drive/
/content/drive/My Drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installa

In [ ]:
import os
import json
import random
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch_geometric.data import Data

import SimpleITK as sitk
from skimage.measure import label, regionprops

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import timm
import networkx as nx

# **2. Importación del dataset**

⚠️ <font color='magenta'>Modificar por su ruta personal para: **IA2_Copia_de_Consolidado_**</font>

In [ ]:
# Ubicación del dataset
dataDir = "/content/drive/My Drive/Proyecto_IA2/Dataset/IA2_Copia_de_Consolidado_/"

In [ ]:
# Importación del dataset
if os.path.exists(dataDir): # Verifica si el directorio existe
    directories = [d for d in os.listdir(dataDir) if os.path.isdir(os.path.join(dataDir, d))]
    print("Directories in the lecture-only drive:")
    for directory in directories:
        print(directory) # Imprime la lista de nombres de los subdirectorios (id de paciente)
else:
    print("The lecture drive path does not exist.")

Directories in the lecture-only drive:
10297_1000303
10118_1000118
10707_1000723
11065_1001087
10201_1000205
10370_1000376
10941_1000960
11309_1001332
10642_1000658
10147_1000149
11119_1001142
10071_1000071
11412_1001436
11234_1001257
10475_1000483
11093_1001116
10481_1000489
10502_1000511
10469_1000477
11199_1001222
11224_1001247
10228_1000232
10339_1000345
11081_1001103
10168_1000171
11402_1001426
11298_1001321
10880_1000896
10634_1000649
10061_1000061
10717_1000733
10793_1000809
10951_1000970
10924_1000941
11372_1001395
10493_1000502
10360_1000366
10211_1000215
10922_1000939
10196_1000200
10580_1000593
10582_1000596
11073_1001095
10345_1000351
11307_1001330
11374_1001397
11075_1001097
10108_1000108
10038_1000038
10762_1000778
10833_1000849
10063_1000063
11117_1001140
11115_1001138
10491_1000500
10213_1000217
11400_1001424
10953_1000972
11222_1001245
11156_1001179
10926_1000943
10705_1000721
10943_1000962
10936_1000953
10347_1000353
10467_1000475
11197_1001220
10547_1000558
10580_100

# **3. Se reduce el dataset a los que tienen centroide**

⚠️ <font color='magenta'>Modificar por su ruta personal para: **IA2_8x64x64-CIspheres.json**</font>

In [ ]:
# Ruta del archivo JSON
json_path = "/content/drive/My Drive/Proyecto_IA2/Dataset/IA2_8x64x64-CIspheres.json"

In [ ]:
from collections import Counter

with open(json_path, 'r') as file:
    data = json.load(file)

label_counts = Counter()
binary_label_counts = Counter()

for directory in directories:
  idx = 0
  while True:
    directory_json = f"{directory}_{idx:03d}"
    if directory_json in data:
      lesion_info = data[directory_json]
      label = lesion_info.get("label")
      if label is not None:
        label_counts[label] += 1
        binary_label = 0 if label == 0 else 1
        binary_label_counts[binary_label] += 1
      idx += 1
    else:
      break

# Total de etiquetas
total_labels = sum(label_counts.values())
total_binary = sum(binary_label_counts.values())

# Crear DataFrames ordenados por etiqueta y agregar porcentaje
df_labels = pd.DataFrame(sorted(label_counts.items()), columns=["Etiqueta", "Cantidad"])
df_labels["Porcentaje (%)"] = (df_labels["Cantidad"] / total_labels * 100).round(2)

df_binary = pd.DataFrame(sorted(binary_label_counts.items()), columns=["Etiqueta (Binaria)", "Cantidad"])
df_binary["Porcentaje (%)"] = (df_binary["Cantidad"] / total_binary * 100).round(2)

# Mostrar resultados sin índice
print("Tabla de etiquetas originales:")
print(df_labels.to_string(index=False))

print("\nTabla de etiquetas binarizadas:")
print(df_binary.to_string(index=False))

Tabla de etiquetas originales:
 Etiqueta  Cantidad  Porcentaje (%)
        0       847           78.21
        2       143           13.20
        3        54            4.99
        4        20            1.85
        5        19            1.75

Tabla de etiquetas binarizadas:
 Etiqueta (Binaria)  Cantidad  Porcentaje (%)
                  0       847           78.21
                  1       236           21.79


In [ ]:
# Se dejan los pacientes con lesiones que estén en el archivo JSON y tengan centroide

directorio_nuevo = []

with open(json_path, 'r') as file:
    data = json.load(file)

for directory in directories:
    idx = 0
    while True:
        directory_json = f"{directory}_{idx:03d}"
        if directory_json in data:
            lesion_info = data[directory_json]
            centroide = lesion_info.get("centroid")
            label = lesion_info.get("label")
            if centroide is not None:
                label_binaria = 0 if label == 0 else 1
                directorio_nuevo.append({
                    "paciente": directory,
                    "lesion_id": directory_json,
                    "centroid": centroide,
                    "label": label_binaria
                })
            idx += 1
        else:
            break

In [ ]:
print(f"Tamaño del directorio nuevo: {len(directorio_nuevo)}")

Tamaño del directorio nuevo: 1083


# **4. Generación de grafos**

**Creación de puntos por lesión**

In [ ]:
def mostrar_secuencia_con_puntos_sin_mascara(centroide, num_puntos_ext = 7, num_puntos_int = 3, radio_interno_fijo = 10, radio_externo_fijo = 25):

  # Inicializar variables
  centroid_y, centroid_x = centroide[1], centroide[2]

  # Usar radios fijos
  angulos_int = np.linspace(0, 2 * np.pi, num_puntos_int, endpoint=False)
  puntos_internos = [
      (int(centroid_y + radio_interno_fijo * np.sin(a)),
        int(centroid_x + radio_interno_fijo * np.cos(a)))
      for a in angulos_int
  ]

  angulos_ext = np.linspace(0, 2 * np.pi, num_puntos_ext, endpoint=False)
  puntos_externos = [
      (int(centroid_y + radio_externo_fijo * np.sin(a)),
        int(centroid_x + radio_externo_fijo * np.cos(a)))
      for a in angulos_ext
  ]

  # Diccionario de ubicaciones
  ubicaciones = {
      "puntos_internos": puntos_internos,
      "puntos_externos": puntos_externos
  }

  return ubicaciones

**Red preentrenada**

In [ ]:
def cargar_red_preentrenada(output_dim=512):

  model = timm.create_model('resnet34', pretrained=True)

  # Cambiar la primera convolución para imágenes de 1 canal
  model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

  model.global_pool = nn.Identity()

  model.fc = nn.Sequential(
      nn.AdaptiveAvgPool2d((1, 1)),
      nn.Flatten(),
      nn.Linear(512, output_dim)
  )

  model.feature_reduction = model.fc

  model.eval()
  return model

**Otras redes preentrenadas**

In [ ]:
def cargar_red_preentrenada_resnet101(output_dim=512):

  model = timm.create_model('resnet101', pretrained=True)  # Sin features_only
  model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
  model.global_pool = nn.Identity()
  model.fc = nn.Sequential(  # Sustituye la capa final
      nn.AdaptiveAvgPool2d((1, 1)),
      nn.Flatten(),
      nn.Linear(2048, output_dim)
  )
  model.feature_reduction = model.fc
  model.eval()
  return model

In [ ]:
def cargar_red_preentrenada_efficientnet_b0(output_dim=512):

  model = timm.create_model('efficientnet_b0', pretrained=True)

  # Cambiar la primera capa para aceptar 1 canal
  model.conv_stem = nn.Conv2d(1, model.conv_stem.out_channels,
                                kernel_size=model.conv_stem.kernel_size,
                                stride=model.conv_stem.stride,
                                padding=model.conv_stem.padding,
                                bias=False)

  # Reemplazar el "classifier" con una capa personalizada
  in_features = model.classifier.in_features
  model.classifier = nn.Sequential(
      nn.AdaptiveAvgPool2d((1, 1)),
      nn.Flatten(),
      nn.Linear(in_features, output_dim)
  )

  model.feature_reduction = model.classifier
  model.eval()
  return model

In [ ]:
def cargar_red_preentrenada_vgg19(output_dim=512):
    from torchvision.models import vgg19
    model = vgg19(pretrained=True)

    # Modificar la primera capa para imágenes en escala de grises (1 canal)
    model.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)

    # Congelar los pesos si no vas a entrenar la red
    for param in model.parameters():
        param.requires_grad = False

    # Reemplazar la cabeza de clasificación por una capa de reducción de características
    model.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, output_dim)
    )

    # Envolvemos la extracción de features como método propio
    def forward_features(x):
        x = model.features(x)
        x = torch.flatten(x, 1)
        return x

    model.forward_features = forward_features
    model.feature_reduction = model.classifier

    model.eval()
    return model

**Preprocesamiento de parches**

In [ ]:
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.astype(np.float32) / 65535.0),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

**Extracción de parches**

In [ ]:
def extraer_parche(sequence_layer, punto, patch_size=32):

  y, x = punto
  half = patch_size // 2

  y_min = max(y - half, 0)
  y_max = min(y + half, sequence_layer.shape[0])
  x_min = max(x - half, 0)
  x_max = min(x + half, sequence_layer.shape[1])

  patch = np.zeros((patch_size, patch_size), dtype=sequence_layer.dtype)
  patch_crop = sequence_layer[y_min:y_max, x_min:x_max]

  y_offset = half - (y - y_min)
  x_offset = half - (x - x_min)

  patch[y_offset:y_offset + patch_crop.shape[0], x_offset:x_offset + patch_crop.shape[1]] = patch_crop

  return patch

**Extraer features de un parche**

In [ ]:
def extraer_features_parche(sequence_layer, punto, model, device='cpu', patch_size=32):

  patch = extraer_parche(sequence_layer, punto, patch_size)

  input_tensor = transform(patch).unsqueeze(0).to(device)  # (1, 1, 224, 224)

  with torch.no_grad():
      feat_map = model.forward_features(input_tensor)  # (1, 2048, 7, 7)
      features = model.feature_reduction(feat_map)     # (1, 512)
      features = features.view(-1)                     # (512,)

  return features.cpu().numpy()

**Extraer features de múltiples puntos**

In [ ]:
def extraer_features_nodos(sequence_layer, puntos, model, device='cpu'):

  features_nodos = []

  for punto in puntos:
    features = extraer_features_parche(sequence_layer, punto, model, device)
    features_nodos.append(features)

  return np.stack(features_nodos)

**Reducción de dimensionalidad**

In [ ]:
def reducir_dimensionalidad(features, n_components=100):

  n_samples, n_features = features.shape
  n_components = min(n_components, n_samples, n_features)

  pca = PCA(n_components=n_components)
  features_reducidas = pca.fit_transform(features)

  return features_reducidas

**Construcción del grafo de una lesión**

In [ ]:
def construir_grafo(nodos_internos, nodos_externos):

  G = nx.Graph()
  total_nodos = nodos_internos.shape[0] + nodos_externos.shape[0]

  # Se agregan los nodos
  for idx in range(total_nodos):
    G.add_node(idx)

  # Se conectan los nodos internos entre sí
  for i in range(len(nodos_internos)):
    for j in range(i + 1, len(nodos_internos)):
      G.add_edge(i, j)

  # Se conectan cada nodo externo con cada nodo interno
  for ext_idx in range(len(nodos_externos)):
    externo_id = len(nodos_internos) + ext_idx
    for int_idx in range(len(nodos_internos)):
      G.add_edge(externo_id, int_idx)

  return G

**Conversión del grafo a PyG Data**

In [ ]:
def convertir_a_data(G, features, label, lesion_id):

  edge_index = torch.tensor(list(G.edges)).t().contiguous()
  x = torch.tensor(features, dtype=torch.float)
  y = torch.tensor([label], dtype=torch.long)

  data = Data(x=x, edge_index=edge_index, y=y)
  data.lesion_id = lesion_id  # Se añade el atributo adicional

  return data

**Procesamiento de una lesión**

In [ ]:
def procesar_lesion(lesion, dataDir, model, device='cpu'):

  centroid = lesion["centroid"]
  paciente = lesion["paciente"]

  # Rutas de imágenes
  t2w_path = f"{dataDir}{paciente}/{paciente}_0000.nii.gz"
  adc_path = f"{dataDir}{paciente}/{paciente}_0001.nii.gz"
  dwi_path = f"{dataDir}{paciente}/{paciente}_0002.nii.gz"

  # Se cargan las imágenes
  t2w_img = sitk.GetArrayFromImage(sitk.ReadImage(t2w_path))
  adc_img = sitk.GetArrayFromImage(sitk.ReadImage(adc_path))
  dwi_img = sitk.GetArrayFromImage(sitk.ReadImage(dwi_path))

  t2w_layer = t2w_img[centroid[0], :, :]
  adc_layer = adc_img[centroid[0], :, :]
  dwi_layer = dwi_img[centroid[0], :, :]

  # Se obtienen los nodos internos y externos
  nodos = mostrar_secuencia_con_puntos_sin_mascara(centroid)
  internos = nodos["puntos_internos"]
  externos = nodos["puntos_externos"]

  # Se extraen features de los puntos
  t2w_internos = extraer_features_nodos(t2w_layer, internos, model, device)
  adc_internos = extraer_features_nodos(adc_layer, internos, model, device)
  dwi_internos = extraer_features_nodos(dwi_layer, internos, model, device)

  t2w_externos = extraer_features_nodos(t2w_layer, externos, model, device)
  adc_externos = extraer_features_nodos(adc_layer, externos, model, device)
  dwi_externos = extraer_features_nodos(dwi_layer, externos, model, device)

  # Se concatenan y se reduce la dimensionalidad
  internos_features = np.concatenate([t2w_internos, adc_internos, dwi_internos], axis=1)
  externos_features = np.concatenate([t2w_externos, adc_externos, dwi_externos], axis=1)

  all_features = np.vstack((internos_features, externos_features))

  return {
      "features": all_features,
      "n_internos": len(internos),
      "n_externos": len(externos),
      "label": lesion["label"]
  }

**Creación del dataset de grafos**

In [ ]:
def crear_dataset_completo(directorio_nuevo, dataDir, device='cpu'):

  model = cargar_red_preentrenada().to(device)

  all_features_global = []
  info_lesiones = []

  for lesion in directorio_nuevo:
    resultado = procesar_lesion(lesion, dataDir, model, device)
    all_features_global.append(resultado["features"])
    info_lesiones.append({
        "n_internos": resultado["n_internos"],
        "n_externos": resultado["n_externos"],
        "label": resultado["label"],
        "lesion_id": lesion["lesion_id"]  # Se guarda el ID de la lesión
    })

  # PCA Global
  all_features_concat = np.vstack(all_features_global)
  pca = PCA(n_components=100)
  all_features_reducidas = pca.fit_transform(all_features_concat)

  # Dividir en grafos
  dataset_grafos = []
  idx = 0

  for info in info_lesiones:
    total_nodos = info["n_internos"] + info["n_externos"]
    features_lesion = all_features_reducidas[idx:idx + total_nodos]
    idx += total_nodos

    G = construir_grafo(
        np.zeros((info["n_internos"],)),
        np.zeros((info["n_externos"],))
    )

    data = convertir_a_data(G, features_lesion, info["label"], info["lesion_id"])
    dataset_grafos.append(data)

  return dataset_grafos

**Manejo del sistema**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_grafos = crear_dataset_completo(directorio_nuevo, dataDir, device)

# **5. Inspección del dataset**

In [ ]:
def inspeccionar_dataset(dataset_grafos):

  print(f"Número total de grafos: {len(dataset_grafos)}")

  # Se revisan las características de algunos grafos aleatorios
  indices_muestra = random.sample(range(len(dataset_grafos)), 10)
  print(f"\n Mostrando detalles de grafos en índices: {indices_muestra}")

  for idx in indices_muestra:
    data = dataset_grafos[idx]
    print(f"ID Lesión: {data.lesion_id}")
    num_nodos = data.x.size(0)
    num_features = data.x.size(1)
    num_aristas = data.edge_index.size(1)
    etiqueta = data.y.item()

    print(f"Grafo {idx}:")
    print(f" - Número de nodos: {num_nodos}")
    print(f" - Número de características: {num_features}")
    print(f" - Número de aristas: {num_aristas}")
    print(f" - Etiqueta: {etiqueta}")
    print("-" * 40)

# Uso:
inspeccionar_dataset(dataset_grafos)

Número total de grafos: 1083

 Mostrando detalles de grafos en índices: [987, 549, 529, 429, 913, 164, 446, 1005, 940, 113]
ID Lesión: 10747_1000763_000
Grafo 987:
 - Número de nodos: 10
 - Número de características: 100
 - Número de aristas: 24
 - Etiqueta: 0
----------------------------------------
ID Lesión: 10888_1000904_000
Grafo 549:
 - Número de nodos: 10
 - Número de características: 100
 - Número de aristas: 24
 - Etiqueta: 1
----------------------------------------
ID Lesión: 10997_1001016_000
Grafo 529:
 - Número de nodos: 10
 - Número de características: 100
 - Número de aristas: 24
 - Etiqueta: 0
----------------------------------------
ID Lesión: 11136_1001159_000
Grafo 429:
 - Número de nodos: 10
 - Número de características: 100
 - Número de aristas: 24
 - Etiqueta: 0
----------------------------------------
ID Lesión: 10097_1000097_000
Grafo 913:
 - Número de nodos: 10
 - Número de características: 100
 - Número de aristas: 24
 - Etiqueta: 1
---------------------------

# **6. Guardar dataset**

⚠️ <font color='magenta'>Si desea guardar el dataset de grafos en su Drive, modifique la siguiente ruta por la de la ubicación deseada en su Drive.

💡 También puede modificar el nombre del archivo si lo desea.</font>

In [ ]:
def guardar_dataset(dataset_grafos, nombre_archivo):

  carpeta_destino = "/content/drive/My Drive/Proyecto_IA2/Dataset"
  os.makedirs(carpeta_destino, exist_ok=True)

  ruta_guardado = os.path.join(carpeta_destino, nombre_archivo)

  torch.save(dataset_grafos, ruta_guardado)
  print(f"Dataset guardado en: {ruta_guardado}")

nombre_archivo = 'IA2_DatasetGrafos_PICAI_Consolidado'
guardar_dataset(dataset_grafos, nombre_archivo)

Dataset guardado en: /content/drive/My Drive/Bivlab/Datasets/PICAI_Consolidado/Grafos06_PICAI_Consolidado
